In [118]:
import os
import pandas as pd

In [119]:
df = pd.read_csv("../Datasets/EuroparlNutidsr_trainset_verbs.csv", sep=";")
original_pos = list(df["comment_text"].values)
original_labels = list(df["label"].values)
df

,comment_text,label
0,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
1,PUNCT PRON AUX VERB NOUN DET ADJ NOUN ADP PUNC...,1
2,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
3,NOUN VERB PRON PUNCT ADP ADV DET NOUN NOUN AUX...,0
4,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
...,...,...
2461873,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> PRON VERB ...,1
2461874,<PAD> PRON VERB SCONJ PRON AUX VERB ADV ADP PA...,1
2461875,AUX VERB ADV ADP PART VERB NOUN CCONJ ADP NOUN...,0
2461876,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0


In [133]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
correct_sentences = lines[-5000:]
correct_sentences = [line.strip("\n") for line in correct_sentences]
len(correct_sentences)

5000

In [123]:
import pandas as pd
import os
import torch
from transformers import Trainer, BertTokenizer
import numpy as np
import torch.nn.utils.prune as prune


os.chdir("/Users/lucasvilsen/Desktop/GrammatiktakDatasets/Danish")
print(os.listdir())

test_sentences_verbs = pd.read_csv("present_tense.csv", sep="|")
correct_sentences = test_sentences_verbs["correct"].values
len(correct_sentences)

['spelling_errors.csv', 'README.md', 'danish_ner.pickle', 'present_tense.csv']


244

In [134]:
correct_sentences[0]

'Liikanen, om De hurtigt vil afklare dette med Deres kollega, hr.'

In [135]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/")

import pickle
import stanza
from tqdm import tqdm
import os



def get_pos(x):
    if os.path.exists(f"FineTuneModels/cache/pos_caching_{len(x)}.pkl"):
        print("pos_caching.pkl already exists")
        with open(f"FineTuneModels/cache/pos_caching_{len(x)}.pkl", "rb") as f:
            pos_list = pickle.load(f)
    else: 
        pos_list = []
        pos_tagger = stanza.Pipeline("da", processors='tokenize,pos', use_gpu=True, cache_directory='./cache', tokenize_pretokenized=True, n_process=4)
        for sentence in tqdm(x):
            pos = get_pos_tags(sentence, pos_tagger)
            pos_list.append(pos)
        print(len(pos_list))
        print("Updating")
        with open(f"FineTuneModels/cache/pos_caching_{len(x)}.pkl", "wb") as f:
            pickle.dump(pos_list, f)
        print("Updated")
    return pos_list

def get_pos_tags(sentence, pos_tagger):
    doc = pos_tagger(sentence)
    features = [word.feats if word.feats else None for sentence in doc.sentences for word in sentence.words]
    feature_dicts = turn_features_to_dicts(features)
    results = [(word.upos, [word.start_char, word.end_char], feature_dicts[i]) for sentence in doc.sentences for i, word in enumerate(sentence.words)]
    return results

def turn_features_to_dicts(features):
    feature_dicts = []
    current_tense = None
    for feature in features:
        if feature is None:
            feature_dicts.append({})
            continue
        feature_dict = {}
        current_features = feature.split("|")
        for current_feature in current_features:
            key, value = current_feature.split("=")
            if key == "Tense" and current_tense is None:
                current_tense = value
            feature_dict[key] = value
        if "Tense" not in feature_dict and "VerbForm" in feature_dict and key is not None:
            feature_dict["Tense"] = "Pres" if current_tense is None else current_tense
        feature_dicts.append(feature_dict)
    return feature_dicts

all_pos = get_pos(correct_sentences)

pos_caching.pkl already exists


In [236]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r_bøjninger.pickle", "rb") as f:
    nutids_r_bøjninger = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

from tqdm import tqdm

testset = []
labels = []

padded_words = []

padding_left = 3
padding_right = 1

def get_pos_tags(index):
    current_pos = all_pos[index]
    return [current_pos[i][0] for i in range(len(current_pos))]

og_index = 0
comma_right_before_index = 0
at_indexes = []
at_index = -1


for y in tqdm(range(len(correct_sentences))):
    line = correct_sentences[y]
    if len(str(line)) < 1 or str(line) == "nan":
        continue
    line = line.strip("\n")
    true_words = line.split()
    pos = get_pos_tags(y)
    words = ["<PAD>"]*padding_left + pos + ["<PAD>"]*padding_right
    true_padded_words = ["<PAD>"]*padding_left + true_words + ["<PAD>"]*padding_right
    for i, word in enumerate(true_words):
        try: stemmed = nutids_r_stem[word]
        except: continue
        if word[-1] == "s" or words[i+padding_left] != "VERB":
            continue
        if true_words[i-1].lower().strip() == "og": 
            og_index += 1
            continue
        if true_words[i-1][-1] == ",":
            comma_right_before_index += 1
            continue
        at_index += 1
        if true_words[i-1].lower().strip() == "at": 
            at_indexes.append(at_index)
        if nutids_r_bøjninger[stemmed][0] == word:
            labels.append(1)
        else:
            labels.append(0)
        testset.append(" ".join(words[i:i+padding_left+padding_right+1]))
        padded_words.append(" ".join(true_padded_words[i:i+padding_left+padding_right+1]))

100%|██████████| 5000/5000 [00:00<00:00, 46066.35it/s]


In [237]:
new_testset = [" ".join(sent_pos.split()) for sent_pos in testset]
new_pos_and_labels = [(" ".join(sent_pos.split()[15-padding_left:16+padding_right]), label) for sent_pos, label in zip(original_pos, original_labels)]
new_pos = [x[0] for x in new_pos_and_labels]

In [238]:
duplicates_dict = {}
duplicates = []
for element in tqdm(new_pos_and_labels):
    if element[0] in duplicates_dict:
        if element[1] != duplicates_dict[element[0]]:
            duplicates.append(element[0])
    else:
        duplicates_dict[element[0]] = element[1]

100%|██████████| 2461878/2461878 [00:00<00:00, 2715312.06it/s]


In [239]:
len(set(duplicates)), len(duplicates_dict), len(set(new_pos))

(6011, 17755, 17755)

In [240]:
print(len(new_pos), len(original_pos))
print(new_testset[0])
print(new_pos[0])

from tqdm import tqdm
can_guess = 0

predictions = []

for sent_pos in tqdm(new_testset):
    if sent_pos in duplicates_dict and sent_pos not in duplicates:
        can_guess += 1
        predictions.append(duplicates_dict[sent_pos])
    else:
        predictions.append(None)

can_guess 

2461878 2461878
PRON ADV AUX VERB PRON
<PAD> <PAD> PRON VERB NOUN


100%|██████████| 5921/5921 [00:16<00:00, 349.38it/s]


1820

In [241]:
len(predictions), len(labels)

(5921, 5921)

In [242]:
import numpy as np

a = np.array(predictions)
b = np.array(labels)

no_guess = 0
correct = 0
wrong = 0

for p, l in zip(predictions, labels):
    if p is None:
        no_guess += 1
        continue
    if p == l:
        correct += 1
    else:
        wrong += 1
    
no_guess = no_guess / len(predictions) * 100
correct = correct / len(predictions) * 100
wrong = wrong / len(predictions) * 100

print("Correct: ", round(correct, 2), "%")
print("Wrong: ", round(wrong, 2), "%")
print("No guess: ", round(no_guess, 2), "%")
print("Correct to wrong ratio: ", round(correct/wrong, 2))

Correct:  30.64 %
Wrong:  0.1 %
No guess:  69.26 %
Correct to wrong ratio:  302.33


In [255]:
from tabulate import tabulate

correct = [30, 53, 64, 63, 53, 41, 30, 23]
ratio = [302, 196, 123, 114, 137, 202, 137, 139]
procentwise_ratio = [round(el/302, 2) for el in ratio]
final_score = [round(x/100 + y, 2) for x,y in zip(correct, procentwise_ratio)]

print("Correct: ", correct)
print("Ratios: ", ratio)
print("Final combined score: ", final_score)


Correct:  [30, 53, 64, 63, 53, 41, 30, 23]
Ratios:  [302, 196, 123, 114, 137, 202, 137, 139]
Final combined score:  [1.3, 1.18, 1.05, 1.01, 0.98, 1.08, 0.75, 0.69]


Thereby the best dict method is:

1. 3-1
2. 4-1
3. 8-1
4. 5-1